In [137]:
import pandas
import json


In [138]:
length_of = 5

In [139]:
template_path = "../data/output/template.json"
template = json.load(open(template_path))
template.keys()

dict_keys(['total', 'supplier', 'storages', 'buildings', 'areas', 'line'])

In [140]:
systems_path = "../data/systems_optional.json"
systems = json.load(open(systems_path))
systems.keys()

dict_keys(['supplier', 'storages', 'line'])

In [141]:
arnis_path = "../data/total_Arnis.json"
arnis = json.load(open(arnis_path))
arnis.keys()

dict_keys(['highways', 'areas', 'buildings'])

In [142]:
highway_coordinates = {}

In [143]:
def generate_coordinate_id(coordinate):
    return f"lat<{coordinate["lat"]}>lon<{coordinate["lon"]}>"

In [144]:
from objects import Coordinate

In [145]:
buildings = arnis["buildings"]
buildings_coordinates = {}
for buildingId, building in buildings.items():
    geometry = building["geometry"]
    for coordinate in geometry:
        buildings_coordinates[generate_coordinate_id(coordinate)] = Coordinate.deserialize(coordinate)

In [146]:
arnis_output = template.copy()

In [147]:
import random

In [148]:
usage = [random.randint(0, 100) for _ in range(length_of)]

In [149]:
arnis_output["buildings"] = {}
for buildingId, building in buildings.items():
    arnis_output["buildings"][buildingId] = {}
    arnis_output["buildings"][buildingId]["supplier"] = {}
    for systemKey, system in systems["supplier"].items():
        arnis_output["buildings"][buildingId]["supplier"][systemKey] = {
            "usage": usage,
            "totalInvest": random.randint(0, 10000),
            "operatingCost": random.randint(0, 10000),
            "totalCo2": random.randint(0, 10000),
            "totalArea": random.randint(0, 10),
        }

    demands = {
        "Electro": usage,
        "Heating": usage,
    }

    arnis_output["buildings"][buildingId]["demands"] = demands

In [150]:
areas = arnis["areas"]
areas_coordinates = {}
for areaId, area in areas.items():
    geometry = area["geometry"]
    for coordinate in geometry:
        areas_coordinates[generate_coordinate_id(coordinate)] = Coordinate.deserialize(coordinate)

In [151]:
arnis_output["areas"] = {}
for areaId, area in areas.items():
    arnis_output["areas"][areaId]= {}
    arnis_output["areas"][areaId]["supplier"] = {}
    for systemKey, system in systems["supplier"].items():
        arnis_output["areas"][areaId]["supplier"][systemKey] = {
            "usage": usage,
            "totalInvest": random.randint(0, 10000),
            "operatingCost": random.randint(0, 10000),
            "totalCo2": random.randint(0, 10000),
            "totalArea": random.randint(0, 10),
        }

In [152]:
highways = arnis["highways"]
highways_coordinates = {}
for highwayId, highway in highways.items():
    geometry = highway["geometry"]
    for coordinate in geometry:
        highways_coordinates[generate_coordinate_id(coordinate)] = Coordinate.deserialize(coordinate)

In [153]:
lines = systems["line"]
lines = [line for line in lines.values()]

In [154]:
arnis_output["line"] = {}
pipes = []
for highwayId, highway in highways.items():
    # with a chance of 50% we will skip
    if random.randint(0, 1) == 0:
        continue

    # choose a random line
    line = random.choice(lines)
    kind = line["name"]
    used_nodes = []
    connections = []

    coordinates = highway["geometry"]
    for coordinate in coordinates:
        coordinate_id = generate_coordinate_id(coordinate)
        used_nodes.append(coordinate_id)

    for idx, coordinate in enumerate(coordinates):
        if idx == len(coordinates) - 1:
            break
        connections.append({
            "from": generate_coordinate_id(coordinates[idx]),
            "to": generate_coordinate_id(coordinates[idx + 1]),
            "capacity": random.randint(0, 100),
            "usage": usage,
            "loss": [0 for _ in range(length_of)],
        })

    pipe = {
        "kind": kind,
        "connections": connections,
        "usedNodes": used_nodes,
        "totalInvest": random.randint(0, 10000),
        "operatingCost": random.randint(0, 10000),
        "totalCo2": random.randint(0, 10000),
    }

    pipes.append(pipe)

arnis_output["line"] = pipes


In [155]:
with open("../data/output/arnis_random.json", "w") as f:
    json.dump(arnis_output, f)

In [156]:
arnis_output["supplier"]

{'Photovoltaic': {'totalInvest': 0,
  'operatingCost': 0,
  'totalCo2': 0,
  'totalArea': 0}}

In [157]:
import re

pattern = re.compile(r"lat<([+-]?[0-9]*[.]?[0-9]+)>lon<([+-]?[0-9]*[.]?[0-9]+)>")

solution = arnis_output
system_file = "./../data/systems_optional.json"
with open(system_file, "r") as file:
    systems = json.load(file)

suppliers_options = systems["supplier"]
pipe_options = systems["line"]

areas = solution["areas"]
pipes = solution["line"]

In [159]:
buildings = solution["buildings"]
buildingSupplierRender = {}
supplierTypes = set()
orgBuildings = arnis["buildings"]

supplierRenders = {}
for supplierType in suppliers_options:
    coordinateLat = []
    coordinateLon = []
    hoverText = []
    for buildingId, building in buildings.items():
        suppliers = building["supplier"]
        if supplierType not in suppliers:
            continue
        orgBuilding = orgBuildings[buildingId]
        geometry = orgBuilding["geometry"]
        most_north_east = geometry[0]
        for coordinate in geometry:
            if coordinate["lat"] > most_north_east["lat"] and coordinate["lon"] > most_north_east["lon"]:
                most_north_east = coordinate
                
        coordinateLat.append(most_north_east["lat"])
        coordinateLon.append(most_north_east["lon"])
        hoverText.append(supplierType)
        
    for areaId, area in areas.items():
        suppliers = area["supplier"]
        if supplierType not in suppliers:
            continue
            
        orgArea = arnis["areas"][areaId]
        geometry = orgArea["geometry"]
        most_north_east = geometry[0]
        for coordinate in geometry:
            if coordinate["lat"] > most_north_east["lat"] and coordinate["lon"] > most_north_east["lon"]:
                most_north_east = coordinate
        
        coordinateLat.append(most_north_east["lat"])
        coordinateLon.append(most_north_east["lon"])
        hoverText.append(supplierType)
        
    if len(coordinateLat) == 0:
        continue
        
    color = suppliers_options[supplierType]["color"]
    supplierRenders[supplierType] = {
        "lat": coordinateLat,
        "lon": coordinateLon,
        "color": color,
        "hoverText": hoverText,
    }

In [37]:
buildings.keys()

dict_keys(['supplier', 'demands'])